# Exercice 2 –PCA avec Scikit-Learn sur le jeu de données «iris »

## Question 1 : Charger le jeu de données «iris»

Lire  le  dataset  iris  en  utilisant  la  méthode  datasets.load_iris()  de  sklearn.datasets. Explorer ce datase

In [1]:
from Ex1 import *
from sklearn.datasets import load_iris
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## Question 2 : Réaliser une ACP sur le jeu de données «iris»

Appliquer  l’ACP  à  ce  dataset  en  utilisant  2  composantes  principales.  Vérifier  la dimension des données après application de l’ACP

## Question 3 : les composantes principales

Afficher les composantes principales en utilisant principal.components

## Question 4 : Affichage des données

Afficher les données dans le nouveau repère des composantes principalesen utilisant2 composantes. Afficher chaque classe avec une couleur différente

## Question 5 : proportion de variance expliquée

Calculer et afficher la proportion de variance expliquée (explained_variance_ratio_) pour chaque composante principale

## Question 6 : Dimension minimales requises

Écrire un code Python permettant de calculer le nombre de dimensions requises pour préserver 95% de la variance du jeu de données:
1. Utiliser la méthode cumsum() de numpy
2. Proposer une autre méthode en jouant sur la valeur du paramètre n_components de la classe PCA de Scikit-learn.

## Question 7 : Visualisation de la variance expliquée

Représenter  la  contribution  de  la  variance  en  fonction  du  nombre  de  dimensions (représenter graphiquement cumsum). Interpréter

On remarque que les deux premières composantes principales expliquent plus de 95% de la variance.
Par ailleurs, avec 4 composantes ont arrives à 100% de la variance expliquée.